In [ ]:
import os
import sys
import numpy as np
import pandas as pd

from multiprocessing import Pool

from itertools import product

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

In [ ]:
experiment_results_dir = "../../results/experiment-results"
fold_dir_template = "fold={}"
evaluation_file_template = "Model{}_validation_epoch{:03d}_results.csv"

folds = [1, 2, 3, 4, 5]
model_types = ["CNN", "GCN"]
epochs = [e for e in range(1, 101)]


In [ ]:
idx2anomalyid = {0: 0, 1: 2, 2: 5}
anomalyid2idx = {v:k for k,v in idx2anomalyid.items()}
prob_cols = ["prob_node_anomaly_{}".format(v) for v in idx2anomalyid.values()]

In [ ]:
def get_path(fold, model_type, epoch):
    fold_dir = fold_dir_template.format(fold)
    evaluation_file = evaluation_file_template.format(model_type, epoch)
    path = os.path.join(experiment_results_dir, fold_dir, evaluation_file)
    
    return path

In [ ]:
def calculate_scores(labels, predictions, prefix="mc"):
    scores = {
        f"{prefix}_acc": accuracy_score(labels, predictions),
        f"{prefix}_acc_balanced": balanced_accuracy_score(labels, predictions),
        f"{prefix}_f1_avg": f1_score(labels, predictions, average='macro'),
        f"{prefix}_f1_wavg": f1_score(labels, predictions, average='weighted'),
        f"{prefix}_prec_avg": precision_score(labels, predictions, average='macro'),
        f"{prefix}_prec_wavg": precision_score(labels, predictions, average='weighted'),
        f"{prefix}_rec_avg": recall_score(labels, predictions, average='macro'),
        f"{prefix}_rec_wavg": recall_score(labels, predictions, average='weighted'),
        f"{prefix}_confusion_matrix": confusion_matrix(labels, predictions)
    }
    
    for i, s in enumerate(f1_score(labels, predictions, average=None)):
        scores[f"{prefix}_f1_{idx2anomalyid[i]}"] = s
    for i, s in enumerate(precision_score(labels, predictions, average=None)):
        scores[f"{prefix}_prec_{idx2anomalyid[i]}"] = s
    for i, s in enumerate(recall_score(labels, predictions, average=None)):
        scores[f"{prefix}_rec_{idx2anomalyid[i]}"] = s
    
    return scores

In [ ]:
def run(fold):
    df_scores = pd.DataFrame()
    for model_type, epoch in product(model_types, epochs):
        path = get_path(fold, model_type, epoch)
        df = pd.read_csv(path)
        df["true_graph_anomaly_idx"] = df["true_node_anomaly"].apply(lambda val: val) # simple copy
        df["true_graph_anomaly"] = df["true_node_anomaly"].apply(lambda val: val) # simple copy
        df["prediction"] = df[prob_cols].apply(lambda row: idx2anomalyid[np.argmax(row.to_numpy())], axis=1)
        
        group_df = df[["true_graph_anomaly", "true_graph_anomaly_idx", "prediction", "file_id", "sequence_id"]].groupby(["file_id"]).agg({
            "true_graph_anomaly": lambda col: col.values.ravel()[np.flatnonzero(col.values)].tolist(),
            "true_graph_anomaly_idx": lambda col: np.flatnonzero(col.values).tolist(),
            "prediction": list,
            "sequence_id": max
        })
                
        graph_true_labels = group_df["true_graph_anomaly"].values.tolist()
        graph_true_label_indices = group_df["true_graph_anomaly_idx"].values.tolist()
        node_pred_labels = group_df["prediction"].values.tolist()
        sequence_ids = group_df["sequence_id"].values.tolist()
        file_ids = group_df.index.tolist()
                        
        labels = df["true_node_anomaly"].to_numpy()
        predictions = df["prediction"].to_numpy()
        
        # scores for multi-class
        mc_scores = calculate_scores(labels, predictions, prefix="mc")
        
        # scores for binary-class
        labels[labels > 0] = 1
        predictions[predictions > 0] = 1
        bc_scores = calculate_scores(labels, predictions, prefix="bc")
        
        scores = {**mc_scores, **bc_scores}
        
        scores["fold"] = fold
        scores["model_type"] = model_type
        scores["epoch"] = epoch
        scores["graph_true_labels"] = graph_true_labels
        scores["graph_true_label_indices"] = graph_true_label_indices
        scores["node_pred_labels"] = node_pred_labels
        scores["file_ids"] = file_ids
        scores["sequence_ids"] = sequence_ids

        df_scores = df_scores.append(scores, ignore_index=True)
    return df_scores

In [ ]:
with Pool(len(folds)) as p:
    dfs_scores = p.map(run, folds)

df_scores = pd.concat(dfs_scores, ignore_index=True)
print(len(df_scores))
df_scores.to_csv(f"../../results/calculated_scores.csv", index=False)